In [2]:
import psycopg2 as ps
import pandas as pd

In [3]:
conn = ps.connect(
    host='localhost',
    port='5432',
    dbname='northwind',
    user='northwind',
    password='password')

# Top 10 countries based on the revenue

In [4]:
df = pd.read_sql_query('''
    SELECT
    	c.country,
    	COUNT(DISTINCT c.customerid) AS num_customer,
        ROUND(SUM(od.quantity * unitprice * (od.discount + 1)), 2) AS revenue
    FROM customers c
    JOIN orders o ON c.customerid = o.customerid
    JOIN order_details od ON o.orderid = od.orderid 
    GROUP BY c.country
    ORDER BY revenue DESC, num_customer DESC
    LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\3785697717.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,country,num_customer,revenue
0,USA,13,281549.35
1,Germany,11,258996.63
2,Austria,2,150989.42
3,Brazil,9,123011.18
4,France,10,89639.20
5,Venezuela,4,64819.15
6,Ireland,1,64654.88
7,Sweden,2,64552.26
8,UK,7,62261.71
9,Canada,3,60471.91


## Insight
1. USA has the highest number of distinct customers (13), followed by Germany (11) and France (10). This suggests that these countries have a relatively broad customer base, contributing to their high revenue.
2. On the other hand, countries like Austria, Ireland and Sweden have a low number of customers (1 and 2, respectively) but still generate substantial revenue. This could indicate that these customers are high-value, making large purchases. This might also suggest a potential for expanding the customer base in these countries, as there is already evidence of strong spending behavior. 

# Performance of different companies based on the number of orders and the revenue generated

In [5]:
df = pd.read_sql_query('''
    SELECT
        c.customerid,
        c.companyName,
        COUNT(DISTINCT o.orderID) AS num_customer,
        ROUND(SUM(od.quantity * od.unitprice * (od.discount + 1)), 2) AS revenue
    FROM customers c
    JOIN orders o ON c.customerid = o.customerid
    JOIN order_details od ON o.orderid = od.orderid 
    GROUP BY c.customerid, c.companyName
    ORDER BY revenue DESC, num_customer DESC
    LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\2766035877.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,customerid,companyname,num_customer,revenue
0,SAVEA,Save-a-lot Markets,31,126984.83
1,QUICK,QUICK-Stop,28,124689.48
2,ERNSH,Ernst Handel,30,121598.38
3,HUNGO,Hungry Owl All-Night Grocers,19,64654.88
4,RATTC,Rattlesnake Canyon Grocery,18,53394.00
5,FOLKO,Folk och f� HB,19,35543.54
6,MEREP,M�re Paillarde,13,35535.61
7,HANAR,Hanari Carnes,14,35360.93
8,QUEEN,Queen Cozinha,13,34734.70
9,KOENE,K�niglich Essen,14,32583.12


# Insight

1. Top Revenue Generators: Save-a-lot Markets (SAVEA), QUICK-Stop (QUICK), and Ernst Handel (ERNSH) lead in revenue, with high order volumes and significant purchase values, making them key customers.

2. High-Value Orders: Companies like Hungry Owl All-Night Grocers (HUNGO) generate substantial revenue with fewer orders, indicating high-value transactions, possibly due to premium offerings.

3. Growth Opportunities: Mid-tier companies such as Rattlesnake Canyon Grocery (RATTC) and Queen Cozinha (QUEEN) have moderate order counts but lower revenues, suggesting potential to increase average order value through upselling or product diversification.

# Employees ranked based on their total sales and calculates the difference between each employee's sales and the top sales

In [6]:
df = pd.read_sql_query('''
    SELECT 
        e.employeeID,
        e.employeeName AS employee_name,
        SUM(od.unitPrice * od.quantity * (1 - od.discount)) AS total_sales,
        RANK() OVER (ORDER BY SUM(od.unitPrice * od.quantity * (1 - od.discount)) DESC) AS sales_rank,
        SUM(od.unitPrice * od.quantity * (1 - od.discount)) - 
        FIRST_VALUE(SUM(od.unitPrice * od.quantity * (1 - od.discount))) OVER (ORDER BY SUM(od.unitPrice * od.quantity * (1 - od.discount)) DESC) AS sales_difference_from_top
    FROM employees e
    JOIN orders o ON e.employeeID = o.employeeID
    JOIN order_details od ON o.orderID = od.orderID
    GROUP BY e.employeeID, e.employeeName
    ORDER BY sales_rank;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\3422920874.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,employeeid,employee_name,total_sales,sales_rank,sales_difference_from_top
0,4,Margaret Peacock,232890.8460,1,0.0000
1,3,Janet Leverling,202812.8430,2,-30078.0030
2,1,Nancy Davolio,192107.6045,3,-40783.2415
3,2,Andrew Fuller,166537.7550,4,-66353.0910
4,8,Laura Callahan,126862.2775,5,-106028.5685
5,7,Robert King,124568.2350,6,-108322.6110
6,9,Anne Dodsworth,77308.0665,7,-155582.7795
7,6,Michael Suyama,73913.1295,8,-158977.7165
8,5,Steven Buchanan,68792.2825,9,-164098.5635


# Insight

1. Top Performer: Margaret Peacock leads significantly with $232,890.85 in sales, setting a high benchmark for others.

2. Performance Gaps: There's a noticeable decline in sales after the top 3 performers, with gaps of $30,078.00 and more from the top, especially as we move down the ranks.

3. Improvement Opportunities: The lower-tier employees show a substantial gap from the top, indicating a need for targeted support and strategies to boost their performance.

# Shipper performance

In [7]:
df = pd.read_sql_query('''
    SELECT 
        s.shipperid,
        s.companyName,
        COUNT(DISTINCT o.orderid) AS orders_delivered,
        SUM(CASE WHEN o.requireddate >= o.shippeddate THEN 1 ELSE 0 END) AS orders_delivered_on_time,
        ROUND(
            (COUNT(DISTINCT o.orderid) - SUM(CASE WHEN o.requireddate >= o.shippeddate THEN 1 ELSE 0 END))::numeric 
            / COUNT(DISTINCT o.orderid) * 100, 
            2
        ) AS failure_rate
    FROM orders o 
    JOIN shippers s ON o.shipperid = s.shipperid
    GROUP BY s.shipperid
    ORDER BY orders_delivered DESC;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\2869311.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,shipperid,companyname,orders_delivered,orders_delivered_on_time,failure_rate
0,2,United Package,326,299,8.28
1,3,Federal Shipping,255,240,5.88
2,1,Speedy Express,249,233,6.43


# Insight
1. Delivery Volume: United Package has the highest delivery volume with 326 orders, followed by Federal Shipping with 255 orders, and Speedy Express with 249 orders.

2. On-Time Delivery Performance: All three shippers have strong on-time delivery rates, with United Package delivering 299 out of 326 orders on time, but Federal Shipping leads in minimizing the failure rate at 5.88%.

3. Failure Rate Analysis: United Package has the highest failure rate at 8.28%, indicating room for improvement in on-time delivery, whereas Federal Shipping has the lowest failure rate, suggesting better consistency in meeting delivery deadlines.

# Average delivery days and total orders for each shipper, based on the difference between the shipped date and order date.

In [8]:
df = pd.read_sql_query('''
    SELECT 
        o.shipperID,
        ROUND(AVG(o.shippedDate - o.orderDate), 2) AS average_delivery_days,
        COUNT(o.orderID) AS total_orders
    FROM orders o
    WHERE o.shippedDate IS NOT NULL AND o.orderDate IS NOT NULL
    GROUP BY o.shipperID
    ORDER BY average_delivery_days ASC;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\1277848431.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,shipperid,average_delivery_days,total_orders
0,3,7.47,249
1,1,8.57,245
2,2,9.23,315


# Insight
1. Fastest Shipper: Federal Shipping (Shipper ID 3) has the shortest average delivery time at 7.47 days, making it the most efficient among the shippers.

2. Moderate Efficiency: Speedy Express (Shipper ID 1) has an average delivery time of 8.57 days, slightly slower but still competitive.

3. Room for Improvement: United Package (Shipper ID 2) has the longest average delivery time at 9.23 days, despite handling the most orders (315), indicating a potential need for process optimization to improve efficiency.

# Top 10 revenue-generating products that are currently not discontinued

In [9]:
df = pd.read_sql_query('''
    SELECT
        p.productid,
        p.productname,
        ROUND(SUM(od.quantity * od.unitprice), 2) AS revenue
    FROM products p
    JOIN order_details od ON p.productid = od.productid
    WHERE p.discontinued = 0
    GROUP BY p.productid
    ORDER BY revenue DESC
    LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\1262674038.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,productid,productname,revenue
0,38,C�te de Blaye,149984.2
1,59,Raclette Courdavault,76296.0
2,60,Camembert Pierrot,50286.0
3,62,Tarte au sucre,49827.9
4,56,Gnocchi di nonna Alice,45121.2
5,51,Manjimup Dried Apples,44742.6
6,18,Carnarvon Tigers,31987.5
7,72,Mozzarella di Giovanni,25738.8
8,43,Ipoh Coffee,25079.2
9,69,Gudbrandsdalsost,24307.2


# Insight
1. Top Revenue Generator: Côte de Blaye leads significantly with a revenue of $149,984.20, far surpassing the other products, making it a critical product for the business.

2. Diverse Product Success: The top 10 products represent a diverse range of categories, from cheese (Raclette Courdavault, Camembert Pierrot) to specialty items (Tarte au sucre, Gnocchi di nonna Alice), indicating that revenue is being driven by a variety of product types.

3. Revenue Disparity: There's a noticeable drop in revenue from Côte de Blaye to the next highest product, Raclette Courdavault ($76,296.00), suggesting that Côte de Blaye is an outlier, and efforts might be focused on boosting the sales of other high-potential products to balance the revenue

# Analyzes the sales performance of products over three consecutive years (2013, 2014, and 2015). It calculates the sales growth rate between 2013-2014 and 2014-2015 and filters for products with consistent growth across these years

In [10]:
df = pd.read_sql_query('''
    WITH subq AS (
        SELECT 
            p.productid,
            p.productname,
            SUM(
                CASE 
                    WHEN EXTRACT(YEAR FROM o.orderdate) = 2013
                    THEN od.quantity * od.unitprice
                    ELSE 0
                END
            ) AS year_2013_sales,
            SUM(
                CASE 
                    WHEN EXTRACT(YEAR FROM o.orderdate) = 2014
                    THEN od.quantity * od.unitprice
                    ELSE 0
                END
            ) AS year_2014_sales,
            SUM(
                CASE 
                    WHEN EXTRACT(YEAR FROM o.orderdate) = 2015
                    THEN od.quantity * od.unitprice
                    ELSE 0
                END
            ) AS year_2015_sales
        FROM products p
        JOIN order_details od ON p.productid = od.productid
        JOIN orders o ON od.orderid = o.orderid
        GROUP BY p.productid
    )
    SELECT 
        productid,
        productname,
        year_2013_sales,
        year_2014_sales,
        year_2015_sales,
        CASE 
            WHEN year_2014_sales != 0 
            THEN ROUND((year_2014_sales - year_2013_sales)::numeric / year_2014_sales * 100, 2)
            ELSE NULL
        END AS growth_rate_13_14,
        CASE 
            WHEN year_2015_sales != 0 
            THEN ROUND((year_2015_sales - year_2014_sales)::numeric / year_2015_sales * 100, 2)
            ELSE NULL
        END AS growth_rate_14_15
    FROM subq
    WHERE year_2015_sales >= year_2014_sales AND year_2014_sales >= year_2013_sales
    ORDER BY growth_rate_14_15 DESC, growth_rate_13_14 DESC
    LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\983718520.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,productid,productname,year_2013_sales,year_2014_sales,year_2015_sales,growth_rate_13_14,growth_rate_14_15
0,13,Konbu,432.0,848.4,3954.0,49.08,78.54
1,37,Gravad lax,603.2,676.0,1768.0,10.77,61.76
2,67,Laughing Lumberjack Lager,56.0,910.0,1596.0,93.85,42.98
3,6,Grandma's Boysenberry Spread,720.0,2500.0,4125.0,71.20,39.39
4,34,Sasquatch Ale,1008.0,2240.0,3430.0,55.00,34.69
5,57,Ravioli Angelo,2074.8,2281.5,3451.5,9.06,33.90
6,32,Mascarpone Fabioli,1331.2,3136.0,4704.0,57.55,33.33
7,24,Guarana Fantastica,568.8,1756.8,2457.0,67.62,28.50
8,1,Chai,1800.0,5295.6,7182.0,66.01,26.27
9,49,Maxilaku,1920.0,3240.0,4340.0,40.74,25.35


# Insight
1. Top Growth Performer: Konbu (Product ID 13) leads in sales growth, with an impressive 78.54% increase from 2014 to 2015 and a 49.08% increase from 2013 to 2014, indicating strong and accelerating demand.

2. Significant Growth Trends: Products like Gravad lax and Laughing Lumberjack Lager show consistent and substantial growth across both periods, making them key products to watch for continued success.

3. Sustained Sales Momentum: Chai and Mascarpone Fabioli maintain strong sales growth over the three years, with notable increases in both 2013-2014 and 2014-2015, suggesting these products have a stable and growing customer base.

# Monthly order statistics, including total orders, average order value, running average order value, total sales, and cumulative (running) total sales, and orders the results by month in descending order.

In [11]:
df = pd.read_sql_query('''
    SELECT 
        TO_CHAR(o.orderdate, 'YYYY-MM') AS months,
        COUNT(DISTINCT o.orderid) AS total_orders,
        ROUND(AVG(od.quantity * od.unitprice * (1 - od.discount)), 2) AS average_order_value,
        ROUND(AVG(AVG(od.quantity * od.unitprice * (1 - od.discount)))
            OVER (ORDER BY TO_CHAR(o.orderdate, 'YYYY-MM')), 2) AS running_average,
        ROUND(SUM(od.quantity * od.unitprice * (1 - od.discount)), 2) AS total_sales,
        ROUND(SUM(SUM(od.quantity * od.unitprice * (1 - od.discount))) 
            OVER (ORDER BY TO_CHAR(o.orderdate, 'YYYY-MM')), 2) AS running_total
    FROM orders o
    JOIN order_details od ON o.orderid = od.orderid
    GROUP BY months
    ORDER BY months DESC;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\290802510.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,months,total_orders,average_order_value,running_average,total_sales,running_total
0,2015-05,14,310.74,567.31,18333.63,1265793.04
1,2015-04,74,687.77,578.98,123798.68,1247459.41
2,2015-03,73,589.07,573.80,104854.16,1123660.73
3,2015-02,54,814.88,573.03,99415.29,1018806.57
4,2015-01,55,619.88,560.30,94222.11,919391.28
5,2014-12,48,626.30,556.99,71398.43,825169.17
6,2014-11,34,489.14,552.92,43533.81,753770.75
7,2014-10,38,629.71,556.90,66749.23,710236.94
8,2014-09,37,585.57,552.05,55629.24,643487.71
9,2014-08,33,562.95,549.65,47287.67,587858.47


# Insights
1. Sales Momentum: There is a noticeable increase in total sales and average order value from mid-2013 to early 2015, with significant peaks in early 2015, indicating a strong upward trend in business performance.

2. Fluctuations in Order Value: Although the overall trend shows growth, there are fluctuations in average order value across different months. For example, February 2015 has the highest average order value at $814.88, indicating possible seasonal demand or specific high-value orders during that period.

3. Running Averages: The running average and running total columns show a steady increase over time, reflecting consistent overall growth in sales, despite the month-to-month variations in individual metrics.




# Calculates the total revenue generated by each product category and then determines the percentage of total sales that each category contributes.

In [12]:
df = pd.read_sql_query('''
    WITH category_sales AS (
        SELECT 
            cat.categoryID,
            cat.categoryName,
            SUM(od.unitPrice * od.quantity * (1 - od.discount)) AS total_revenue
        FROM categories cat
        JOIN products p ON cat.categoryID = p.categoryID
        JOIN order_details od ON p.productID = od.productID
        GROUP BY cat.categoryID, cat.categoryName
    ),
    total_sales AS (
        SELECT 
            SUM(total_revenue) AS overall_revenue
        FROM category_sales
    )
    SELECT 
        cs.categoryName,
        cs.total_revenue,
        ROUND((cs.total_revenue / ts.overall_revenue) * 100, 2) AS percentage_of_total_sales
    FROM category_sales cs, total_sales ts
    ORDER BY cs.total_revenue DESC;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\4193929319.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,categoryname,total_revenue,percentage_of_total_sales
0,Beverages,267868.1800,21.16
1,Dairy Products,234507.2850,18.53
2,Confections,167357.2250,13.22
3,Meat & Poultry,163022.3595,12.88
4,Seafood,131261.7375,10.37
5,Condiments,106047.0850,8.38
6,Produce,99984.5800,7.90
7,Grains & Cereals,95744.5875,7.56


# Insights
1. Dominant Categories: "Beverages" and "Dairy Products" are the top two revenue-generating categories, accounting for 21.16% and 18.53% of total sales, respectively. These categories are key drivers of the business's overall revenue.

2. Mid-Level Performers: "Confections," "Meat & Poultry," and "Seafood" contribute moderately to the revenue, with percentages ranging from 10% to 13%. These categories are significant but not as dominant as the top two.

3. Opportunities for Growth: The lower-performing categories like "Condiments," "Produce," and "Grains & Cereals," which contribute between 7.56% and 8.38% of total sales, could be areas to explore for potential growth or optimization strategies.

# Identifies customers who have not placed any orders in the last six months by comparing their most recent order date with orders within the last six months.

In [13]:
df = pd.read_sql_query('''
    WITH RecentOrderDates AS (
        SELECT 
            customerID,
            MAX(orderdate) AS most_recent_orderdate
        FROM orders
        GROUP BY customerID
    ),
    OrdersInLast6Months AS (
        SELECT 
            c.customerID
        FROM customers c
        JOIN orders o ON c.customerID = o.customerID
        JOIN RecentOrderDates r ON c.customerID = r.customerID
        WHERE o.orderdate > r.most_recent_orderdate - INTERVAL '6 Months'
    )
    SELECT 
        c.customerID,
        c.companyName
    FROM customers c
    LEFT JOIN OrdersInLast6Months o ON c.customerID = o.customerID
    WHERE o.customerID IS NULL
    ORDER BY c.customerID;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\2247964003.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,customerid,companyname
0,FISSA,FISSA Fabrica Inter. Salchichas S.A.
1,PARIS,Paris sp�cialit�s


# Insights
1. Customer Retention Concern: The output lists customers who haven't ordered in the past six months, indicating potential issues with customer retention that may need to be addressed.

2. Target for Re-engagement: These customers are prime candidates for re-engagement strategies, such as marketing campaigns or special offers, to encourage them to return.

3. Business Health Indicator: A large number of customers in this list could signal a declining customer base or satisfaction, prompting a need for deeper analysis into the reasons behind their inactivity.


# Identifies pairs of products that have been ordered together at least five times, showing the top 10 most frequently ordered product pairs.

In [14]:
df = pd.read_sql_query('''
    SELECT 
        p1.productName AS product_1,
        p2.productName AS product_2,
        COUNT(*) AS times_ordered_together
    FROM order_details od1
    JOIN order_details od2 ON od1.orderID = od2.orderID AND od1.productID < od2.productID
    JOIN products p1 ON od1.productID = p1.productID
    JOIN products p2 ON od2.productID = p2.productID
    GROUP BY p1.productName, p2.productName
    HAVING COUNT(*) >= 5
    ORDER BY times_ordered_together DESC
    LIMIT 10;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\2389568333.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,product_1,product_2,times_ordered_together
0,Sir Rodney's Scones,Sirop d'�rable,8
1,Pavlova,Gorgonzola Telino,7
2,Pavlova,Tarte au sucre,6
3,Gorgonzola Telino,Mozzarella di Giovanni,6
4,Nord-Ost Matjeshering,Tourti�re,6
5,Camembert Pierrot,Flotemysost,6
6,Pavlova,Camembert Pierrot,6
7,Raclette Courdavault,Lakkalik�ri,5
8,Gorgonzola Telino,Manjimup Dried Apples,5
9,Chang,Pavlova,5


# Insights
1. Complementary Products: The output reveals pairs of products that are often purchased together, indicating possible complementary or popular combinations that customers prefer.

2. Cross-Selling Opportunities: Knowing which products are frequently ordered together allows for targeted cross-selling strategies, such as bundling these items or recommending one when the other is added to a cart.

3. Inventory and Stocking Decisions: High-frequency pairings can inform inventory management, ensuring that these complementary products are adequately stocked to meet customer demand and avoid missed sales opportunities.

# Ranks employees based on their efficiency in processing orders, measured by the average number of days between the order date and the shipped date.

In [15]:
df = pd.read_sql_query('''
    SELECT 
        e.employeeName AS employee_name,
        COUNT(o.orderID) AS total_orders_handled,
        ROUND(AVG(o.shippedDate - o.orderDate), 2) AS average_processing_days,
        RANK() OVER (ORDER BY AVG(o.shippedDate - o.orderDate)) AS efficiency_rank
    FROM 
        employees e
    JOIN 
        orders o ON e.employeeID = o.employeeID
    WHERE 
        o.shippedDate IS NOT NULL
        AND o.orderDate IS NOT NULL
    GROUP BY 
        e.employeeName
    ORDER BY 
        efficiency_rank;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\569956905.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,employee_name,total_orders_handled,average_processing_days,efficiency_rank
0,Steven Buchanan,42,7.02,1
1,Nancy Davolio,120,7.76,2
2,Andrew Fuller,93,8.05,3
3,Robert King,69,8.38,4
4,Janet Leverling,127,8.43,5
5,Laura Callahan,100,8.68,6
6,Margaret Peacock,151,8.82,7
7,Michael Suyama,65,9.09,8
8,Anne Dodsworth,42,10.86,9


# Insights:
1. Employee Efficiency: The output highlights which employees are most efficient in handling orders, with lower average processing days indicating better performance.

2. Workload Distribution: The number of orders handled by each employee provides insight into the distribution of workload across the team. For instance, an efficient employee with a higher workload (like Nancy Davolio) might be a key contributor.

3. Performance Improvement: Employees with higher average processing days and lower efficiency ranks could be identified for performance improvement initiatives, training, or workload adjustments to enhance overall team efficiency.

# The total sales amount before discount and the total discount given to each customer, ordering the results by the highest total discount given.

In [16]:
df = pd.read_sql_query('''
    SELECT 
        c.customerID,
        c.companyName,
        SUM(od.unitPrice * od.quantity) AS total_amount_before_discount,
        ROUND(SUM(od.unitPrice * od.quantity * od.discount), 2) AS total_discount_given
    FROM customers c
    JOIN orders o ON c.customerID = o.customerID
    JOIN order_details od ON o.orderID = od.orderID
    GROUP BY c.customerID, c.companyName
    ORDER BY total_discount_given DESC;
''', conn)
df

C:\Users\sethi\AppData\Local\Temp\ipykernel_21520\527401850.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query('''


,customerid,companyname,total_amount_before_discount,total_discount_given
0,SAVEA,Save-a-lot Markets,115673.39,11311.44
1,ERNSH,Ernst Handel,113236.68,8361.70
2,HUNGO,Hungry Owl All-Night Grocers,57317.39,7337.49
3,QUICK,QUICK-Stop,117483.39,7206.09
4,QUEEN,Queen Cozinha,30226.10,4508.60
...,...,...,...,...
84,TORTU,Tortuga Restaurante,10812.15,0.00
85,DUMON,Du monde entier,1615.90,0.00
86,CACTU,Cactus Comidas para llevar,1814.80,0.00
87,OCEAN,Oc�ano Atl�ntico Ltda.,3460.20,0.00


# Insights
1. Top Discounted Customers: The output highlights which customers have received the most discounts, with "Save-a-lot Markets" and "Ernst Handel" receiving the highest total discounts. These customers might be key accounts with significant discount agreements.

2. Customer Value Assessment: By comparing the total amount before discount and the total discount given, you can assess whether the discounts are justified relative to the sales generated by each customer, potentially identifying opportunities to optimize discount strategies.

3. Discount Impact on Revenue: The total discount amounts provide insight into how much revenue is being impacted by discounts, helping evaluate the overall effectiveness and profitability of current discount policies.